## ADAPTED APPLICATION TEMPLATE E-OBS - IN-SITU AIR TEMPERATURE

Package c3s_eqc_automatic_quality_control (https://github.com/bopen/c3s-eqc-automatic-quality-control).
This notebook is a first test for the implementation on the following diagnostics on the 
E-Obs in-situ gridded dataset, chosing as variable of interest Air Temperature:
- Climatology Maps for the variable;
- Time Series of Globally Averaged variable;
As initial test to carry out the diagnostic proposed in the deliverable submitted on September 2022  C3S2_D520.5.1.6_202209_Consolidated_diagnostics_in-situ_v1 
Which can be found in the +Atlantic Teams folder 
WP5 EQC of Reanalysis, Satellite and In-situ Obsv > 1_Deliverables > 09_2022 > 2_Submitted

In [1]:
import warnings
import plotly.express as px
import xarray as xr
import cads_toolbox

from c3s_eqc_automatic_quality_control import diagnostics, download, plot

warnings.filterwarnings("ignore")
cads_toolbox.config.USE_CACHE = True

We create our request to download data in the cache and perform our analyses

In [2]:
collection_id = "insitu-gridded-observations-europe"
request = {
    "product_type": "ensemble_mean",
    "variable": "mean_temperature",
    "grid_resolution": "0.25deg",
    "period": ["1995_2010", "2011_2021"],
    "version": "25.0e",
    "format": "zip",
}

We define an identity function to download data as it is without performing any operation

In [3]:
def identity(ds: xr.Dataset) -> xr.Dataset:
    return ds

In [4]:
monthly_global_temp = download.download_and_transform(
    collection_id,
    request,
    transform_func=identity,
    split_all=True
)

INFO     Gathering file 1 out of 2...

INFO     Gathering file 2 out of 2...

INFO     Aggregating data...

Then we perform the wieghted spatial mean to have the global time series of mean temperature

In [9]:
def spatial_monthly_mean(ds: xr.Dataset) -> xr.Dataset:
    ds = diagnostics.spatial_weighted_mean(ds)
    return ds.resample(time="1M").mean("time")
monthly_global_temp_ts = spatial_monthly_mean(monthly_global_temp)
monthly_global_temp_ts

<xarray.Dataset>
Dimensions:  (time: 324)
Coordinates:
  * time     (time) datetime64[ns] 1995-01-31 1995-02-28 ... 2021-12-31
Data variables:
    tg       (time) float64 dask.array<chunksize=(1,), meta=np.ndarray>
Attributes:
    E-OBS_version:  25.0e
    Conventions:    CF-1.4
    References:     http://surfobs.climate.copernicus.eu/dataaccess/access_eo...
    history:        Mon Mar 28 14:52:46 2022: ncks --no-abc -d time,16436,222...
    NCO:            netCDF Operators version 4.7.5 (Homepage = http://nco.sf....

In [11]:
monthly_global_temp_ts = monthly_global_temp_ts.squeeze()
fig = px.line(
    x=monthly_global_temp_ts["time"],
    y=monthly_global_temp_ts["tg"],
)
fig.update_layout(
    xaxis_title=r"$\mbox{Time [months]}$",
    yaxis_title=r"$\mbox{Temperature}~[^oC]$",
    title="GLOBAL MONTHLY MEAN TEMPERATURE 1995-2021",
)
fig.show()

In [15]:
fig.write_image('E-Obs_Global_timeseries_1995-2021.png')

### Now the monthly climatology maps
We create a function to group the data into months and perform the averages calculating the climatology for the period of interest:

In [12]:
def monthly_climatology(ds: xr.Dataset) -> xr.Dataset:
    ds_month = ds.groupby('time.month').mean('time')
    return ds_month

In [14]:
clim_ds = monthly_climatology(monthly_global_temp)
var = clim_ds['tg'].squeeze()
m = int((var.min()/10).round(2))*10
M = int((var.max()/10).round(2))*10
print(m, M)

-20 30


In [20]:
import plotly.graph_objects as go
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
i=0
#for i in range(len(var.month)):
fig = go.Figure(data =
                    go.Contour(
                        x = var.longitude,
                        y = var.latitude,
                        z = var.isel(month=i).values,
                        colorscale='YlGnBu_r',
                        colorbar=dict(
                            title='°C', # title here
                            titleside='right',
                            titlefont=dict(
                                size=14,
                                family='Arial, sans-serif')
                        ),
                        contours=dict(
                            start=m,
                            end=M,
                            size=1,
    )))
fig.update_layout(
        title={
            'text': r'TEMPERATURE '+months[i]+' Climatology 1995-2021',
            'y':0.9,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
        xaxis_title='longitude [degrees_east]',
        yaxis_title='latitude [degrees_north]',
)
fig.write_image('E-OBS_climatology_map_temp_1995-2021_'+months[i]+'.png')
fig.show()